In [ ]:
%matplotlib inline
import random
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dqnroute.utils import *

from sklearn.model_selection import train_test_split
from cycler import cycler

In [ ]:
target_cols = get_target_cols(10)
neighbors_cols = get_neighbors_cols(10)
addr_cols = get_addr_cols(10)
dst_cols = get_dst_cols(10)
amatrix_cols = get_amatrix_cols(10)
left_cols = ['time', 'pkg_id']+neighbors_cols+amatrix_cols+target_cols
new_cols = ['dst', 'addr'] + left_cols

In [ ]:
data = pd.read_csv('../logs/data_generated2_new.csv', names=new_cols)

In [ ]:
import gc
gc.collect()

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
%load_ext autoreload
%autoreload 2

from dqnroute.networks import *
from functools import partial
from tqdm import tqdm_notebook

In [ ]:
def qnetwork_batches(net, data, batch_size=32):
    n = net.graph_size
    target_cols = get_target_cols(n)
    data_cols = ['addr', 'dst', get_neighbors_cols(n)]
    for (tag, dim) in net.add_inputs:
        if tag == 'amatrix':
            data_cols.append(get_amatrix_cols(n))
        else:
            data_cols.append(mk_num_list(tag + '_', n))

    for (a, b) in make_batches(data.shape[0], batch_size):
        batch = data[a:b]
        inputs = tuple(torch.tensor(batch[cols].values, dtype=torch.float)
                       for cols in data_cols)
        output = torch.tensor(batch[target_cols].values, dtype=torch.float)
        yield (inputs, output)

def qnetwork_pretrain_epoch(net, optimizer, data, **kwargs):
    loss_func = nn.MSELoss()
    for (batch, target) in qnetwork_batches(net, data, **kwargs):
        optimizer.zero_grad()
        output = net(*batch)
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
        yield float(loss)
        
def qnetwork_pretrain(net, data, optimizer='rmsprop', epochs=1,
                      save_net=True, **kwargs):
    optimizer = get_optimizer(optimizer, kwargs)(net.parameters())
    epochs_losses = []
    
    for i in tqdm_notebook(range(epochs)):
        sum_loss = 0
        loss_cnt = 0
        for loss in tqdm_notebook(qnetwork_pretrain_epoch(net, optimizer, data, **kwargs),
                                  desc='epoch {}'.format(i)):
            sum_loss += loss
            loss_cnt += 1
        epochs_losses.append(sum_loss / loss_cnt)
        
    if save_net:
        net.save()
    
    return epochs_losses

def shuffle(df):
    return df.reindex(np.random.permutation(df.index))

# Feed-forward сети

In [ ]:
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [ ]:
QNetworkAmatrix = partial(QNetwork, additional_inputs=[{'tag': 'amatrix'}])

ff_network_amatrix = QNetworkAmatrix(10, activation='tanh')
ff_network_amatrix_64_3 = QNetworkAmatrix(10, layers=[64, 64, 64], activation='tanh')
ff_network_amatrix_128_2 = QNetworkAmatrix(10, layers=[128, 128], activation='tanh')
ff_network_amatrix_32_2 = QNetworkAmatrix(10, layers=[32, 32], activation='tanh')
ff_network_amatrix_32_3 = QNetworkAmatrix(10, layers=[32, 32, 32], activation='tanh')

In [ ]:
ff_network_amatrix_losses = qnetwork_pretrain(ff_network_amatrix, shuffle(data), epochs=10)

In [ ]:
ff_network_amatrix_64_3_losses = qnetwork_pretrain(ff_network_amatrix_64_3, shuffle(data), epochs=10)

In [ ]:
ff_network_amatrix_128_2_losses = qnetwork_pretrain(ff_network_amatrix_128_2, shuffle(data), epochs=10)

In [ ]:
ff_network_amatrix_32_2_losses = qnetwork_pretrain(ff_network_amatrix_32_2, shuffle(data), epochs=10)

In [ ]:
ff_network_amatrix_32_3_losses = qnetwork_pretrain(ff_network_amatrix_32_3, shuffle(data), epochs=10)

In [ ]:
losses_layers = np.array([ff_network_amatrix_losses,
                          ff_network_amatrix_32_2_losses,
                          ff_network_amatrix_32_3_losses,
                          ff_network_amatrix_64_3_losses,
                          ff_network_amatrix_128_2_losses]).transpose()

losses_layers_df = pd.DataFrame(data=losses_layers, columns=['64x2', '32x2', '32x3', '64x3', '128x2'])
losses_layers_df.to_csv('../logs/pre_train_data/layers_comparison.csv')

In [ ]:
losses_layers_df = pd.read_csv('../logs/pre_train_data/layers_comparison.csv')

ff_network_amatrix_losses = losses_layers_df['64x2']
ff_network_amatrix_32_2_losses = losses_layers_df['32x2']
ff_network_amatrix_32_3_losses = losses_layers_df['32x3']
ff_network_amatrix_64_3_losses = losses_layers_df['64x3']
ff_network_amatrix_128_2_losses = losses_layers_df['128x2']

x = range(2, 11)
fsize = 16
plt.figure(figsize=(13, 7))
plt.plot(x, ff_network_amatrix_losses[1:], label='64x2 layers')
plt.plot(x, ff_network_amatrix_32_2_losses[1:], label='32x2 layers')
plt.plot(x, ff_network_amatrix_32_3_losses[1:], label='32x3 layers')
plt.plot(x, ff_network_amatrix_128_2_losses[1:], label='128x2')
plt.plot(x, ff_network_amatrix_64_3_losses[1:], label='64x3')
plt.legend(prop={'size': fsize})
plt.xlabel('Epoch', fontsize=fsize)
plt.xticks(x)
plt.grid()
plt.ylabel('MSE', fontsize=fsize)
plt.title('Comparison of FF network configurations by pre-training speed', fontsize=fsize)
plt.show()

In [ ]:
ff_network_simple = QNetwork(10)
ff_network_amatrix = QNetworkAmatrix(10)

In [ ]:
data_full_network = data[data['pkg_id'] < 5000]

In [ ]:
ff_network_simple_losses = qnetwork_pretrain(ff_network_simple, shuffle(data_full_network), epochs=20)

In [ ]:
ff_network_amatrix_losses = qnetwork_pretrain(ff_network_amatrix, shuffle(data), epochs=20)

In [ ]:
ff_network_amatrix_adam_losses = qnetwork_pretrain(ff_network_amatrix, shuffle(data),
                                                   optimizer='adam', epochs=20, save_net=False)

In [ ]:
ff_network_amatrix_adagrad_losses = qnetwork_pretrain(ff_network_amatrix, shuffle(data),
                                                      optimizer='adagrad', epochs=20, save_net=False)

In [ ]:
ff_network_amatrix_adadelta_losses = qnetwork_pretrain(ff_network_amatrix, shuffle(data),
                                                       optimizer='adadelta', epochs=20, save_net=False)

In [ ]:
losses = np.array([ff_network_amatrix_losses, ff_network_amatrix_adam_losses,
                     ff_network_amatrix_adagrad_losses, ff_network_amatrix_adadelta_losses]).transpose()

In [ ]:
losses_df = pd.DataFrame(data=losses, columns=['rmsprop', 'adam', 'adagrad', 'adadelta'])
losses_df.to_csv('../logs/pre_train_data/optimizer_comparison.csv')

In [ ]:
losses_df = pd.read_csv('../logs/pre_train_data/optimizer_comparison.csv')

ff_network_amatrix_losses = losses_df['rmsprop'][:10]
ff_network_amatrix_adam_losses = losses_df['adam'][:10]
ff_network_amatrix_adagrad_losses = losses_df['adagrad'][:10]
ff_network_amatrix_adadelta_losses = losses_df['adadelta'][:10]

x = range(1, 11)
fsize = 25
plt.figure(figsize=(13, 14))
plt.plot(x, ff_network_amatrix_losses, label='RMSProp')
plt.plot(x, ff_network_amatrix_adam_losses, label='Adam')
plt.plot(x, ff_network_amatrix_adagrad_losses, label='AdaGrad')
plt.plot(x, ff_network_amatrix_adadelta_losses, label='AdaDelta')
plt.legend(prop={'size': fsize})
plt.xlabel('Номер эпохи', fontsize=fsize)
plt.xticks(x)
plt.grid()
plt.ylabel('MSE', fontsize=fsize)
plt.title('Сравнение качества предобучения с разными\n алгоритмами оптимизации (ReLU слои)', fontsize=fsize)
plt.show()

In [ ]:
ff_network_tanh_amatrix = QNetworkAmatrix(10, activation='tanh')

In [ ]:
ff_network_tanh_amatrix_losses = qnetwork_pretrain(ff_network_tanh_amatrix, shuffle(data), epochs=10, save_net=False)

In [ ]:
ff_network_tanh_amatrix_adam_losses = qnetwork_pretrain(ff_network_tanh_amatrix, shuffle(data), epochs=10,
                                                        optimizer='adam', save_net=False)

In [ ]:
ff_network_tanh_amatrix_adagrad_losses = qnetwork_pretrain(ff_network_tanh_amatrix, shuffle(data), epochs=10,
                                                           optimizer='adagrad', save_net=False)

In [ ]:
ff_network_tanh_amatrix_adadelta_losses = qnetwork_pretrain(ff_network_tanh_amatrix, shuffle(data), epochs=10,
                                                            optimizer='adadelta', save_net=False)

In [ ]:
losses_tanh = np.array([ff_network_tanh_amatrix_losses, ff_network_tanh_amatrix_adam_losses,
                        ff_network_tanh_amatrix_adagrad_losses, ff_network_tanh_amatrix_adadelta_losses]).transpose()
losses_tanh_df = pd.DataFrame(data=losses_tanh, columns=['rmsprop', 'adam', 'adagrad', 'adadelta'])
losses_tanh_df.to_csv('../logs/pre_train_data/optimizer_comparison_tanh.csv')

In [ ]:
color_c = cycler('color', ['k'])
style_c = cycler('linestyle', ['-', '--', ':', '-.'])
markr_c = cycler('marker', ['', '.', 'o'])
c_cms = color_c * markr_c * style_c

losses_tanh_df = pd.read_csv('../logs/pre_train_data/optimizer_comparison_tanh.csv')

ff_network_tanh_amatrix_losses = losses_tanh_df['rmsprop']
ff_network_tanh_amatrix_adam_losses = losses_tanh_df['adam']
ff_network_tanh_amatrix_adagrad_losses = losses_tanh_df['adagrad']
ff_network_tanh_amatrix_adadelta_losses = losses_tanh_df['adadelta']

x = range(1, 11)
fsize = 14
ticksize = 10
lw=3
f = plt.figure(figsize=(7, 5))
plt.plot(x, ff_network_tanh_amatrix_losses, label='RMSProp', linewidth=lw, alpha=0.6)#, color='k', linestyle='-')
plt.plot(x, ff_network_tanh_amatrix_adam_losses, label='Adam', linewidth=lw, alpha=0.6)#, color='k', linestyle='--')
plt.plot(x, ff_network_tanh_amatrix_adagrad_losses, label='AdaGrad', linewidth=lw)#, color='k', linestyle=':')
plt.plot(x, ff_network_tanh_amatrix_adadelta_losses, label='AdaDelta', linewidth=lw)#, color='k', linestyle='-.')
plt.legend(prop={'size': 14})
plt.xlabel('Epoch', fontsize=fsize)
plt.xticks(x)
plt.rc('xtick', labelsize=ticksize)
plt.rc('ytick', labelsize=ticksize)
plt.grid()
plt.ylabel('MSE', fontsize=fsize)
plt.ylim(0, 1)
plt.title('Comparison of optimization algorithms by pre-training speed', fontsize=fsize)
plt.show()

plt.savefig("../img/opt-algos-pre-training-comparison.pdf", bbox_inches='tight')

# Конвейеры

In [ ]:
conveyor_network_ng_no_inp = QNetwork(22, scope='conveyor_test_ng', activation='tanh')
conveyor_network_ng_amatrix = QNetwork(22, scope='conveyor_test_ng', activation='tanh',
                                       additional_inputs=[{'tag': 'amatrix'}])
conveyor_network_ng_full = QNetwork(22, scope='conveyor_test_ng', activation='tanh',
                                    additional_inputs=[{'tag': 'amatrix'}, {'tag':'work_status'}])

In [ ]:
data_conv_ng = pd.read_csv('../logs/data_conveyor_gen_energy_test.csv', names=['belt_id']+get_conveyor_data_cols(22))

In [ ]:
data_conv_ng.head()

In [ ]:
conveyor_network_ng_no_inp_losses = qnetwork_pretrain(conveyor_network_ng_no_inp, shuffle(data_conv_ng), epochs=5)

In [ ]:
conveyor_network_ng_amatrix_losses = qnetwork_pretrain(conveyor_network_ng_amatrix, shuffle(data_conv_ng), epochs=10)

In [ ]:
conveyor_network_ng_full_losses = qnetwork_pretrain(conveyor_network_ng_full, shuffle(data_conv_ng), epochs=10)

In [ ]:
conveyor_network_ng_no_inp_losses

In [ ]:
conveyor_network_ng_amatrix_losses

In [ ]:
conveyor_network_ng_full_losses